In [1]:
import numpy as np
import pickle

from inv_kin_body2foot import *
from robot import *

In [2]:
with open('foot_infos', 'rb') as foot_infos_file:
    foot_infos = pickle.load(foot_infos_file)

In [3]:
n = np.array([[0], [0], [-1]])
s = np.array([[-1], [0], [0]])
a = np.array([[0], [1], [0]])
R = np.block([n, s, a])

for fidx in range(len(foot_infos)):
    statel = foot_infos[fidx].foot_left[np.ix_([0, 2, 4]), :].reshape(3, -1)
    stater = foot_infos[fidx].foot_right[np.ix_([0, 2, 4]), :].reshape(3, -1)
    
    num_idx = statel.shape[1]
    joints_left = np.zeros((6, num_idx))
    joints_right = np.zeros((6, num_idx))
    trans_mat_left = np.zeros((4, 4, num_idx))
    trans_mat_right = np.zeros((4, 4, num_idx))
    
    for sidx in range(num_idx):
        p = statel[:, sidx].reshape(3, -1)
        trans_mat = np.block([[R, p],[0, 0, 0, 1]])
        is_left = True
        q_left = inv_kin_body2foot(trans_mat.copy(), is_left)
        joints_left[:, sidx] = q_left
        trans_mat_left[:, :, sidx] = trans_mat

        p = stater[:, sidx].reshape(3, -1)
        trans_mat = np.block([[R, p], [0, 0, 0, 1]])
        is_left = False
        q_right = inv_kin_body2foot(trans_mat.copy(), is_left)
        joints_right[:, sidx] = q_right
        trans_mat_right[:, :, sidx] = trans_mat
        
    foot_infos[fidx].set_joints_left(joints_left)
    foot_infos[fidx].set_joints_right(joints_right)
    foot_infos[fidx].set_trans_mat_left(trans_mat_left)
    foot_infos[fidx].set_trans_mat_right(trans_mat_right)

In [4]:
with open('foot_joint_infos', 'wb') as foot_joint_infos_file:
    pickle.dump(foot_infos, foot_joint_infos_file)